<a href="https://colab.research.google.com/github/aritanaoya/football_analysis/blob/main/J1LeagueScrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import Module

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from tqdm.notebook import tqdm
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import requests
from bs4 import BeautifulSoup
import time
from tqdm.notebook import tqdm
import re
from urllib.request import urlopen
from contextlib import closing
from operator import attrgetter
from pprint import pprint
from subprocess import DEVNULL
from subprocess import Popen
import time


# Team Dic

In [2]:
TeamDic = {'札幌':'276', '仙台':'238', '鹿島':'120', '浦和':'122', '柏':'132', 'FC東京':'270', '川崎Ｆ':'86', '横浜FM':'124', '横浜FC':'296' , '湘南':'130', '清水':'126', '名古屋':'127', 'Ｇ大阪':'128','Ｃ大阪':'133', '神戸':'136','広島':'129', '徳島':'30116', '福岡':'135', '鳥栖':'269', '大分':'207'}

tmpDic = {'札幌':'276', '仙台':'238', '鹿島':'120', '浦和':'122', '柏':'132', 'FC東京':'270', '川崎Ｆ':'86', '横浜FM':'124', '横浜FC':'296' , '湘南':'130', '清水':'126','名古屋':'127', 'Ｇ大阪':'128','Ｃ大阪':'133', '神戸':'136','広島':'129', '徳島':'30116', '福岡':'135', '鳥栖':'269', '大分':'207'}

tmp2Dic = {'清水':'126', '福岡':'135'}

J12021TeamDic = {'札幌':'sapp', '仙台':'send', '鹿島':'kasm', '浦和':'uraw', '柏':'kasw', 'FC東京':'fctk', '川崎Ｆ':'ka-f', '横浜FM':'y-fm', '横浜FC':'y-fc' , '湘南':'shon', '清水':'shim', '名古屋':'nago', 'Ｇ大阪':'g-os','Ｃ大阪':'c-os', '神戸':'kobe','広島':'hiro', '徳島':'toku', '福岡':'fuku', '鳥栖':'tosu', '大分':'oita'}

def get_keys_from_value(d, val):
    return [k for k, v in d.items() if v == val]

get_keys_from_value(TeamDic,'276')

['札幌']

<select id="comparison1_team"><option value="0">チーム選択</option><option value="276">札幌</option><option value="31219">八戸</option><option value="30673">岩手</option><option value="238">仙台</option><option value="30310">秋田</option><option value="294">山形</option><option value="30674">福島</option><option value="120">鹿島</option><option value="176">水戸</option><option value="30305">栃木</option><option value="30103">群馬</option><option value="122">浦和</option><option value="199">大宮</option><option value="121">千葉</option><option value="132">柏</option><option value="270">FC東京</option><option value="123">東京Ｖ</option><option value="30532">町田</option><option value="86">川崎Ｆ</option><option value="124">横浜FM</option><option value="296">横浜FC</option><option value="30675">YS横浜</option><option value="130">湘南</option><option value="30676">相模原</option><option value="150">甲府</option><option value="30528">松本</option><option value="30535">長野</option><option value="193">新潟</option><option value="30308">富山</option><option value="30526">金沢</option><option value="126">清水</option><option value="131">磐田</option><option value="30677">藤枝</option><option value="31296">沼津</option><option value="127">名古屋</option><option value="30302">岐阜</option><option value="134">京都</option><option value="128">Ｇ大阪</option><option value="133">Ｃ大阪</option><option value="136">神戸</option><option value="30000">鳥取</option><option value="30314">岡山</option><option value="129">広島</option><option value="30851">山口</option><option value="30536">讃岐</option><option value="30116">徳島</option><option value="30148">愛媛</option><option value="30974">今治</option><option value="135">福岡</option><option value="30313">北九州</option><option value="269">鳥栖</option><option value="30531">長崎</option><option value="30303">熊本</option><option value="207">大分</option><option value="31508">宮崎</option><option value="31042">鹿児島</option><option value="30317">琉球</option></select>

# ScrapeData

In [3]:
def scrapeData(homeTeamID,awayTeamID,year):
    url = 'https://www.football-lab.jp/comparison/team/' + year + '/' + homeTeamID + '/' + year + '/' + awayTeamID
    #チーム毎の平均スタッツを取得
    #チャンスビルディングポイント
    df = pd.read_html(url)[12]
    df = df.drop([0,2,4,6,8,10,12,14,16]).drop(df.columns[[0,1,5,6]], axis=1)
    # #チームスタイル
    #指数
    # df2 = pd.read_html(url)[13]
    # df2 = df2.drop([0,2,4,6,8,10,12,14]).drop(df2.columns[[0,1,5,6]], axis=1)
    #チャンス構築率
    df3 = pd.read_html(url)[16]
    df3 = df3.drop([0,2,4,6,8,10,11,13,15,17,19]).drop(df3.columns[[0,1,5,6]], axis=1)
    #ゴール期待値
    df4 = pd.read_html(url)[17]
    df4 = df4.drop([0,2,4,6]).drop(df4.columns[[0,1,5,6]], axis=1)
    #トラッキングデータ
    df5 = pd.read_html(url)[20]
    df5 = df5.drop([0,2,4,6]).drop(df5.columns[[0,1,5,6]], axis=1)

    #他のスタッツ
    df6 = pd.read_html(url)[35]
    df6 = df6.drop([0,2,4,6,8,10,12,14,16,18,20,22,24,26,28,30,32,34]).drop(df6.columns[[0,1,5,6]], axis=1)

    df = pd.concat([df,df3,df4,df5,df6])
    df = df.reset_index(drop=True)
    df = df.rename(columns={'試合平均': 'home','Unnamed: 3': 'Data','試合平均.1': 'away'})
    df = df[['Data', 'home', 'away']]
    df = df.set_index('Data')
    df = df.T

    time.sleep(1)

    #試合結果を取得
    html = requests.get(url)

    html.encoding = "UTF-8"
    soup = BeautifulSoup(html.text, "html.parser")

    texts = soup.find('div', attrs={'class': 'gameTbl'}).text
    texts = re.findall(r'\w+', texts)
    time.sleep(1) 

    return df,texts

# Scrape Season Data

In [4]:
def scrapeSeasonData(TeamNameDic,dfList,year):
    for team in TeamNameDic:
        url = 'https://www.football-lab.jp/' +  TeamNameDic[team] + '/match/' + '?year=' + year
        #表を取得
        df = pd.read_html(url)[0]
        #最後の列を削除
        df = df.drop(38).drop(df.columns[[2,6,7,8,19,20]], axis=1)
        #homeTeam名追加
        df.insert(2,'Team',team)
        #列名を英語に
        df = df.rename(columns={'相手': 'awayTeam','Unnamed: 5':'H or A','チャンス構築率':'CBP','シュート':'shot','シュート成功率':'shot acc','支配率':'possession','攻撃CBP':'ACBP','パスCBP':'PassCBP','奪取P':'InterceptP','守備P':'DP'})
        #Home Awayのチーム並びかえ
        df['Team'].where(df['H or A']=='H', df['awayTeam'] ,inplace = True)
        df['awayTeam'].where(df['H or A']=='H', team ,inplace = True)
        #Home Awayのスコアに修正
        df.insert(4,'awayScore',0)
        df.insert(4,'homeScore',0)
        #Teamがhomeの時のscore入力
        df['homeScore'].where(df['H or A']=='A',df['スコア'].str[:1],inplace = True)
        df['awayScore'].where(df['H or A']=='A',df['スコア'].str[2:3],inplace = True)
        #Teamがawayの時のscore入力
        df['awayScore'].where(df['H or A']=='H',df['スコア'].str[:1],inplace = True)
        df['homeScore'].where(df['H or A']=='H',df['スコア'].str[2:3],inplace = True)
        df = df.drop(df.columns[[6]], axis=1)
        dfList.append(df)
        time.sleep(1) 
    return dfList

In [5]:
tmpJ12021TeamDic = {'札幌':'sapp', '仙台':'send',}
dfList = []
dfList =  scrapeSeasonData(tmpJ12021TeamDic,dfList,'2021')
dfList[0]

,節,開催日,Team,awayTeam,homeScore,awayScore,H or A,AGI,KAGI,CBP,shot,shot acc,possession,ACBP,PassCBP,InterceptP,DP
0,1,2.27,札幌,横浜FC,5,1,H,61,66,20.9%,23,21.7%,54.3%,24.11,18.98,131.07,11.56
1,2,3.6,名古屋,札幌,1,0,A,51,56,6.6%,8,0.0%,54.3%,15.23,13.46,102.32,9.16
2,3,3.10,広島,札幌,2,1,A,53,41,17.8%,21,4.8%,60.2%,20.30,15.49,83.07,12.25
3,5,3.17,浦和,札幌,0,0,A,56,55,16.9%,20,0.0%,58.5%,22.35,16.58,120.84,9.72
4,6,3.20,札幌,神戸,3,4,H,54,54,11.8%,15,20.0%,50.0%,19.43,13.50,112.56,15.64
5,7,4.3,福岡,札幌,1,2,A,55,49,9.4%,13,15.4%,56.2%,18.94,15.33,139.59,18.88
6,8,4.7,FC東京,札幌,2,1,A,64,58,16.2%,16,6.3%,57.9%,19.67,16.27,94.02,8.42
7,9,4.11,札幌,鹿島,2,2,H,53,45,14.4%,17,11.8%,56.4%,18.17,12.60,84.62,11.82
8,10,4.16,札幌,横浜FM,1,3,H,54,49,12.9%,17,5.9%,46.4%,14.97,11.51,128.30,13.12
9,11,4.24,札幌,仙台,2,1,H,52,46,14.3%,17,11.8%,65.7%,22.66,17.94,104.13,13.61


In [ ]:
SeasonData = []
SeasonData =  scrapeSeasonData(J12021TeamDic,SeasonData,'2021')
SeasonData

In [7]:
tmp = SeasonData[0].merge(SeasonData[1], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
tmp2 = SeasonData[0].merge(SeasonData[2], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
tmp3 = tmp.merge(tmp2, how="outer")
tmp3

,節,開催日,Team,awayTeam,homeScore,awayScore,H or A_x,AGI_x,KAGI_x,CBP_x,shot_x,shot acc_x,possession_x,ACBP_x,PassCBP_x,InterceptP_x,DP_x,H or A_y,AGI_y,KAGI_y,CBP_y,shot_y,shot acc_y,possession_y,ACBP_y,PassCBP_y,InterceptP_y,DP_y
0,1,2.27,札幌,横浜FC,5,1,H,61,66,20.9%,23,21.7%,54.3%,24.11,18.98,131.07,11.56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,3.6,名古屋,札幌,1,0,A,51,56,6.6%,8,0.0%,54.3%,15.23,13.46,102.32,9.16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,3.10,広島,札幌,2,1,A,53,41,17.8%,21,4.8%,60.2%,20.30,15.49,83.07,12.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5,3.17,浦和,札幌,0,0,A,56,55,16.9%,20,0.0%,58.5%,22.35,16.58,120.84,9.72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6,3.20,札幌,神戸,3,4,H,54,54,11.8%,15,20.0%,50.0%,19.43,13.50,112.56,15.64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,7,4.3,福岡,札幌,1,2,A,55,49,9.4%,13,15.4%,56.2%,18.94,15.33,139.59,18.88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,8,4.7,FC東京,札幌,2,1,A,64,58,16.2%,16,6.3%,57.9%,19.67,16.27,94.02,8.42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,9,4.11,札幌,鹿島,2,2,H,53,45,14.4%,17,11.8%,56.4%,18.17,12.60,84.62,11.82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,10,4.16,札幌,横浜FM,1,3,H,54,49,12.9%,17,5.9%,46.4%,14.97,11.51,128.30,13.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,11,4.24,札幌,仙台,2,1,H,52,46,14.3%,17,11.8%,65.7%,22.66,17.94,104.13,13.61,A,54,48,9.9%,12,8.3%,34.3%,10.77,8.47,93.93,18.06


In [ ]:
seasonDataList = []
for i in range (len(SeasonData)):
    if(i!=0):
        tmp0 = SeasonData[i].merge(SeasonData[0], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
    if(i!=1):
        tmp1 = SeasonData[i].merge(SeasonData[1], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
    if(i!=2):
        tmp2 = SeasonData[i].merge(SeasonData[2], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
    if(i!=3):
        tmp3 = SeasonData[i].merge(SeasonData[3], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
    if(i!=4):
        tmp4 = SeasonData[i].merge(SeasonData[4], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
    if(i!=5):
        tmp5 = SeasonData[i].merge(SeasonData[5], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
    if(i!=6):
        tmp6 = SeasonData[i].merge(SeasonData[6], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
    if(i!=7):
        tmp7 = SeasonData[i].merge(SeasonData[7], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
    if(i!=8):
        tmp8 = SeasonData[i].merge(SeasonData[8], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
    if(i!=9):
        tmp9 = SeasonData[i].merge(SeasonData[9], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
    if(i!=10):
        tmp10 = SeasonData[i].merge(SeasonData[10], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
    if(i!=11):
        tmp11 = SeasonData[i].merge(SeasonData[11], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
    if(i!=12):
        tmp12 = SeasonData[i].merge(SeasonData[12], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
    if(i!=13):
        tmp13 = SeasonData[i].merge(SeasonData[13], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
    if(i!=14):
        tmp14 = SeasonData[i].merge(SeasonData[14], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
    if(i!=15):
        tmp15 = SeasonData[i].merge(SeasonData[15], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
    if(i!=16):
        tmp16 = SeasonData[i].merge(SeasonData[16], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
    if(i!=17):
        tmp17= SeasonData[i].merge(SeasonData[17], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
    if(i!=18):
        tmp18 = SeasonData[i].merge(SeasonData[18], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
    if(i!=19):
        tmp19 = SeasonData[i].merge(SeasonData[19], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
    if(i!=1 and i!=2):
        tmp = tmp1.merge(tmp2, how="outer")
    if(i!=0 and i!=2):
        tmp = tmp0.merge(tmp2, how="outer")
    if(i!=0 and i!=1):
        tmp = tmp0.merge(tmp1, how="outer")
    if(i!=0 and i!=1 and i!=2):
        tmp = tmp.merge(tmp2, how="outer")
    if(i!=3):
        tmp = tmp.merge(tmp3, how="outer")
    if(i!=4):
        tmp = tmp.merge(tmp4, how="outer")
    if(i!=5):
        tmp = tmp.merge(tmp5, how="outer")
    if(i!=6):
        tmp = tmp.merge(tmp6, how="outer")
    if(i!=7):
        tmp = tmp.merge(tmp7, how="outer")
    if(i!=8):
        tmp = tmp.merge(tmp8, how="outer")
    if(i!=9):
        tmp = tmp.merge(tmp9, how="outer")
    if(i!=10):
        tmp = tmp.merge(tmp10, how="outer")
    if(i!=11):
        tmp = tmp.merge(tmp11, how="outer")
    if(i!=12):
        tmp = tmp.merge(tmp12, how="outer")
    if(i!=13):
        tmp = tmp.merge(tmp13, how="outer")
    if(i!=14):
        tmp = tmp.merge(tmp14, how="outer")
    if(i!=15):
        tmp = tmp.merge(tmp15, how="outer")
    if(i!=16):
        tmp = tmp.merge(tmp16, how="outer")
    if(i!=17):
        tmp = tmp.merge(tmp17, how="outer")
    if(i!=18):
        tmp = tmp.merge(tmp18, how="outer")
    if(i!=19):
        tmp = tmp.merge(tmp19, how="outer")
    tmp = tmp.dropna(how='any')
    if(i!=0):
        seasonData = pd.concat([seasonData,tmp])
    else:
        seasonData = tmp
#.merge(tmp6, how="outer").merge(tmp7, how="outer").merge(tmp8, how="outer").merge(tmp9, how="outer").merge(tmp10, how="outer").merge(tmp11, how="outer").merge(tmp12, how="outer").merge(tmp13, how="outer").merge(tmp14, how="outer").merge(tmp15, how="outer").merge(tmp16, how="outer").merge(tmp17, how="outer").merge(tmp18, how="outer").merge(tmp19, how="outer")
seasonData

In [11]:
seasonData.to_csv("2021SeasonData.csv",encoding = 'utf-8-sig')

In [17]:
csv ='/content/2021SeasonData.csv'
data = pd.read_csv(csv)

In [ ]:
#index列を削除
data = data.drop(data.columns[[0]], axis=1)
data

In [19]:
# Hが上に来る様に並び替え
data = data.sort_values('H or A_x', ascending=False)

In [ ]:
#重複削除
data = data.drop_duplicates(subset=['節','開催日','Team','awayTeam'])

In [ ]:
#節を昇順に並び替え
data = data.sort_values('節')
data

In [23]:
#書き出し
data.to_csv("seasonData_2021.csv",encoding = 'utf-8-sig')